In [85]:
import pandas as pd

In [86]:
data = pd.read_csv('test.csv', sep='|')

/home/jmckenzie/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [87]:
#strip whitespace for the column headers
data.columns = [i.strip(' ') for i in data.columns]

to_keep = ['bet_type', 'period', 'quote_time', 'handicap', 'bookie', 'price1', 'price2', 'price3', 'home_team',
          'away_team', 'match_datetime', 'home_full_time_goals', 'away_full_time_goals']

df = data[to_keep]
df = df.dropna(axis=0, subset=['bet_type'])

In [88]:
#strip whitespace from the bet_type column and then remove the asian handicap bets
def remove_whitespace(entry):
    return entry.strip(' ')

df['bet_type'] = df.bet_type.apply(remove_whitespace)
df = df[df.bet_type != 'AH']

In [167]:
def odds_to_probs(odds):
    return 1.0 / odds

df['overound'] = df.price1.apply(odds_to_probs) + df.price2.apply(odds_to_probs)
df['prob1'] = df.price1.apply(odds_to_probs)
df['prob2'] = df.price2.apply(odds_to_probs)
df['1 - prob1'] = df.price1 - 1

df['prob1_mod'] = (df['prob1'] / df['overound']) * 1.01
df['prob2_mod'] = (df['prob2'] / df['overound']) * 1.01


#df['implied_1'] = df.prob1 / df.overound
#df['implied_2'] = df.prob2 / df.overound

In [168]:
df.head()

,bet_type,period,quote_time,handicap,bookie,price1,price2,price3,home_team,away_team,match_datetime,home_full_time_goals,away_full_time_goals,overound,prob1,prob2,1 - prob1,prob1_mod,prob2_mod
16,OU,FT,C,0.5,bet365,1.03,15.0,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.037540,0.970874,0.066667,0.03,0.945103,0.064897
17,OU,FT,C,1.5,bet365,1.18,4.5,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.069680,0.847458,0.222222,0.18,0.800176,0.209824
18,OU,FT,C,2.5,bet365,1.60,2.3,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.059783,0.625000,0.434783,0.60,0.595641,0.414359
19,OU,FT,C,3.5,bet365,2.50,1.5,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.066667,0.400000,0.666667,1.50,0.378750,0.631250
20,OU,FT,C,4.5,bet365,4.33,1.2,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.064280,0.230947,0.833333,3.33,0.219168,0.790832


In [97]:
from scipy.optimize import minimize_scalar
import math
import numpy as np
from scipy.stats import distributions

#make functions
def expected_goal_num(num, probability):
    'Takes the under number and probability from bookies prices and calculates expected number of goals'
    num = int(num - 0.5)
    def func_to_min(x):
        return (distributions.poisson.cdf(num, x) - probability)**2
    return minimize_scalar(func_to_min, method='Golden')['x']

In [171]:
test_df = df.head(1000)

In [175]:
test_df.loc[:, 'expected_goals_over'] = test_df.apply(lambda x: expected_goal_num(x['handicap'], 1 - x['prob1']), axis=1)
test_df.loc[:, 'expected_goals_under'] = test_df.apply(lambda x: expected_goal_num(x['handicap'], x['prob2']), axis=1)

test_df.loc[:, 'goals_over_mod'] = test_df.apply(lambda x: expected_goal_num(x['handicap'], 1 - x['prob1_mod']), axis=1)
test_df.loc[:, 'goals_under_mod'] = test_df.apply(lambda x: expected_goal_num(x['handicap'], x['prob2_mod']), axis=1)

/home/jmckenzie/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/jmckenzie/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [162]:
#tests

#1) whats the probability of 0 goals when mean is 0.45 - test expected goals function and manual calculation
print(expected_goal_num(0, 0.45))
print(-math.log(0.45))


#2) whats the cumulative probability under 1 for a poisson with mean - 2.85
print(expected_goal_num(1.5, 0.5))
print(math.exp(-2.85)*(1 + 2.85))

0.7985076966634275
0.7985076962177716
1.6783469910575168
0.22270063536812806


In [123]:
test_df.head()

,bet_type,period,quote_time,handicap,bookie,price1,price2,price3,home_team,away_team,match_datetime,home_full_time_goals,away_full_time_goals,overound,prob1,prob2,1 - prob1,expected_goals_over,expected_goals_under
16,OU,FT,C,0.5,bet365,1.03,15.0,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.037540,0.970874,0.066667,0.03,3.536117,2.708050
17,OU,FT,C,1.5,bet365,1.18,4.5,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.069680,0.847458,0.222222,0.18,3.350635,2.852905
18,OU,FT,C,2.5,bet365,1.60,2.3,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.059783,0.625000,0.434783,0.60,3.223601,2.948693
19,OU,FT,C,3.5,bet365,2.50,1.5,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.066667,0.400000,0.666667,1.50,3.211323,2.913217
20,OU,FT,C,4.5,bet365,4.33,1.2,,Sarpsborg 08,Sandnes,2014-05-04 14:30:00+01,2,1,1.064280,0.230947,0.833333,3.33,3.264403,2.890385


In [180]:
#for each home team, for each away team:
#get every instance of a match between them
#flag any games where the expected under goals > expected over goals

def get_discrepancies(df):
    discrepancies = []
    for home_team in df['home_team'].unique():
        for away_team in df['away_team'].unique():
            game_series = df[(df['home_team'] ==  home_team) & (df['away_team'] ==  away_team)]
            if any([i > j for i in game_series['goals_under_mod'] for j in game_series['goals_over_mod']]):
                discrepancies.append((home_team.strip(' '), away_team.strip(' ')))
    return discrepancies

In [151]:
#test for a discrepancy - change the first game - Sarpsborg08 vs Sandnes

test_df.loc[16, 'expected_goals_under'] = 4.0

/home/jmckenzie/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [177]:
test_df.head()

,bet_type,period,quote_time,handicap,bookie,price1,price2,price3,home_team,away_team,...,overound,prob1,prob2,1 - prob1,prob1_mod,prob2_mod,expected_goals_over,expected_goals_under,goals_over_mod,goals_under_mod
16,OU,FT,C,0.5,bet365,1.03,15.0,,Sarpsborg 08,Sandnes,...,1.037540,0.970874,0.066667,0.03,0.945103,0.064897,3.536117,2.708050,2.902295,2.734953
17,OU,FT,C,1.5,bet365,1.18,4.5,,Sarpsborg 08,Sandnes,...,1.069680,0.847458,0.222222,0.18,0.800176,0.209824,3.350635,2.852905,2.995483,2.930169
18,OU,FT,C,2.5,bet365,1.60,2.3,,Sarpsborg 08,Sandnes,...,1.059783,0.625000,0.434783,0.60,0.595641,0.414359,3.223601,2.948693,3.085274,3.039681
19,OU,FT,C,3.5,bet365,2.50,1.5,,Sarpsborg 08,Sandnes,...,1.066667,0.400000,0.666667,1.50,0.378750,0.631250,3.211323,2.913217,3.116054,3.071354
20,OU,FT,C,4.5,bet365,4.33,1.2,,Sarpsborg 08,Sandnes,...,1.064280,0.230947,0.833333,3.33,0.219168,0.790832,3.264403,2.890385,3.198768,3.142190


In [181]:
y = get_discrepancies(test_df)

In [182]:
print(y)

[('Sarpsborg 08', 'Sandnes'), ('Sarpsborg 08', 'Haugesund'), ('Werder Bremen', 'Kaiserslautern'), ('Luzern', 'Aarau'), ('Luzern', 'Young Boys'), ('Freiburg', 'Schalke'), ('Freiburg', 'Eintracht Frankfurt'), ('Freiburg', 'St. Pauli'), ('Freiburg', 'B. Monchengladbach'), ('Holstein Kiel', 'Munich 1860'), ('Greuther Furth', 'FSV Frankfurt'), ('Greuther Furth', 'Bochum'), ('Graafschap', 'PSV'), ('Graafschap', 'Den Bosch'), ('Wolfsburg', 'Bayern Munich'), ('Wolfsburg', 'VfB Stuttgart'), ('Wolfsburg', 'Hertha Berlin'), ('Augsburg', 'Hamburger SV'), ('Augsburg', 'Köln'), ('Dortmund', 'Hoffenheim'), ('Wehen', 'Hansa Rostock'), ('Wehen', 'Stutt. Kickers'), ('Thun', 'Young Boys'), ('Thun', 'Luzern'), ('Thun', 'Basel'), ('Den Bosch', 'Venlo'), ('Viking', 'Haugesund'), ('Waalwijk', 'Maastricht'), ('Waalwijk', 'Sparta Rotterdam'), ('Waalwijk', 'G.A. Eagles'), ('Unterhaching', 'Chemnitzer'), ('Sandhausen', 'Freiburg'), ('Sandhausen', 'Arminia Bielefeld'), ('Sandhausen', 'Bochum'), ('Mainz', 'Bayern 

In [ ]:
#for discrepancies - how much do I need to place on each outcome to return one unit of profit
#figure out about ROI and how I would optimize the ROI 